In [1]:
import os
import glob
import time
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import pandas as pd
import numpy as np
from tqdm import tqdm
import openai
from openai import OpenAI


In [2]:
!pwd

/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/notebooks


In [20]:
print(torch.__version__)

2.7.0


In [5]:
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaIoBaseDownload
# import io
# import os

# # Authenticate
# SCOPES = [
#     'https://www.googleapis.com/auth/drive.metadata.readonly',  # View file metadata
#     'https://www.googleapis.com/auth/drive.readonly'           # Download files
# ]
# flow = InstalledAppFlow.from_client_secrets_file('../credentials.json', SCOPES)
# creds = flow.run_local_server(port=0)
# service = build('drive', 'v3', credentials=creds)

# def download_folder(folder_id, destination):
#     os.makedirs(destination, exist_ok=True)
    
#     # Get all files in folder
#     results = service.files().list(
#         q=f"'{folder_id}' in parents",
#         pageSize=1000,
#         fields="files(id, name)"
#     ).execute()
    
#     # Download each file
#     for item in tqdm(results.get('files', [])):
#         request = service.files().get_media(fileId=item['id'])
#         fh = io.FileIO(os.path.join(destination, item['name']), 'wb')
#         downloader = MediaIoBaseDownload(fh, request)
        
#         done = False
#         while not done:
#             status, done = downloader.next_chunk()
#             print(f"Download {item['name']} {int(status.progress() * 100)}%")

# # Usage
# FOLDER_ID = "17F6BKYkP_tr1nRJJw46TIa2jFVqxg5D6"
# # download_folder(FOLDER_ID, "./data/raw/Flicker8k_Dataset")

In [6]:
import os
from pathlib import Path

# Get current working directory (where notebook is run)
PROJECT_ROOT = Path().absolute()  # or Path.cwd()
DATASET_PATH = PROJECT_ROOT / "data" / "raw" / "Flicker8k_Dataset"

In [7]:
DATASET_PATH

PosixPath('/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/notebooks/data/raw/Flicker8k_Dataset')

In [8]:
DATASET_PATH 

PosixPath('/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/notebooks/data/raw/Flicker8k_Dataset')

In [12]:
0.2*8000

1600.0

In [14]:
# import os
# import requests
# import zipfile
# from tqdm import tqdm

# # Configuration
# DATASET_URL = "https://github.com/username/repo/raw/main/data/flickr8k.zip"  # Hosted elsewhere
# DATA_DIR = os.path.join(os.path.dirname(__file__), "..", "data")  # Project-relative path


# def download_file(url, save_path):
#     """Download with progress bar"""
#     os.makedirs(os.path.dirname(save_path), exist_ok=True)
#     response = requests.get(url, stream=True)
#     total_size = int(response.headers.get('content-length', 0))
    
#     with open(save_path, 'wb') as f, tqdm(
#         desc=os.path.basename(save_path),
#         total=total_size,
#         unit='B',
#         unit_scale=True
#     ) as bar:
#         for chunk in response.iter_content(chunk_size=1024):
#             f.write(chunk)
#             bar.update(len(chunk))

# def setup_data():
#     if not os.path.exists(os.path.join(RAW_DIR, "Flicker8k_Dataset")):
#         print("Downloading dataset...")
#         zip_path = os.path.join(DATA_DIR, "flickr8k.zip")
#         download_file(DATASET_URL, zip_path)
        
#         # Unzip
#         with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#             zip_ref.extractall(RAW_DIR)
#         os.remove(zip_path)
#         print("Dataset ready at:", RAW_DIR)
#     else:
#         print("Dataset already exists")

# if __name__ == "__main__":
#     setup_data()

In [15]:
!pip install send2trash

Python(67046) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# import send2trash  # 

# target_dir = '/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/raw/Flicker8k_Dataset'
# protected = {'train', 'val', 'test'}

# for item in os.listdir(target_dir):
#     if item not in protected:
#         try:
#             send2trash.send2trash(os.path.join(target_dir, item))
#             print(f"Moved to trash: {item}")
#         except Exception as e:
#             print(f"Failed to move {item}: {e}")


In [11]:
from dotenv import load_dotenv
import os

# Load .env from the project root (assuming notebook is in notebooks/)
env_path = os.path.join(os.path.dirname(os.getcwd()), ".env")  # Go up one level
load_dotenv(env_path)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [18]:
import os
import glob
import time
import torch
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from openai import OpenAI
from transformers import CLIPProcessor, CLIPModel

# Configuration
DATASET_PATH = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/raw/Flicker8k_Dataset"
OUTPUT_FILE = "flickr8k_captions.csv"
BATCH_SIZE = 32  # Reduced batch size for better memory management
NUM_CAPTIONS_PER_IMAGE = 3  # Reduced number of captions to save API costs
SAVE_EVERY_N_BATCHES = 1
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")  # Get from environment variable

class CaptionGenerator:
    def __init__(self, dataset_path, output_file, batch_size=32, captions_per_image=3):
        """Initialize the caption generator with configuration parameters"""
        self.dataset_path = dataset_path
        self.output_file = output_file
        self.batch_size = batch_size
        self.captions_per_image = captions_per_image
        self.api_call_count = 0
        self.total_cost_estimate = 0
        
        # Initialize OpenAI client
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        
        # Load CLIP model for scoring captions
        print("Loading CLIP model...")
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        
        # Get image paths and check for previous results
        self._prepare_image_paths()
    
    def _prepare_image_paths(self):
        """Get all image paths and filter out already processed ones"""
        self.image_paths = glob.glob(os.path.join(self.dataset_path, "*.jpg"))
        print(f"Found {len(self.image_paths)} images in {self.dataset_path}")
        
        # Check for previous results
        self.results = []
        self.processed_images = set()
        
        if os.path.exists(self.output_file):
            previous_results = pd.read_csv(self.output_file)
            self.results = previous_results.to_dict('records')
            self.processed_images = set(previous_results['image'].values)
            print(f"Resuming from previous run. Already processed {len(self.processed_images)} images.")
        
        # Filter out already processed images
        self.image_paths = [path for path in self.image_paths 
                           if os.path.basename(path) not in self.processed_images]
        print(f"Remaining images to process: {len(self.image_paths)}")
    
    def generate_captions(self, image_path):
        """Generate captions for a single image using GPT-3.5-Turbo"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that generates diverse image descriptions."},
                    {"role": "user", "content": f"Generate {self.captions_per_image} simple descriptions for an image. Each description should be on a new line."}
                ],
                max_tokens=150,
                temperature=0.7
            )
            
            # Track API usage
            self.api_call_count += 1
            # Estimate cost: ~50 tokens input + ~120 tokens output per call
            call_cost = (50 * 0.0005 / 1000) + (120 * 0.0015 / 1000)
            self.total_cost_estimate += call_cost
            
            # Parse and clean captions
            captions = response.choices[0].message.content.strip().split('\n')
            cleaned_captions = []
            for caption in captions:
                # Remove numbering patterns like "1.", "1)", "[1]", etc.
                cleaned = caption.strip()
                if cleaned and cleaned[0].isdigit() and len(cleaned) > 2 and cleaned[1:3] in ['. ', ') ', '- ', ': ']:
                    cleaned = cleaned[3:].strip()
                cleaned_captions.append(cleaned)
            
            return cleaned_captions
            
        except Exception as e:
            print(f"Error generating captions: {e}")
            # Return basic fallback captions
            return ["A photo of a scene", "An image", "A picture"]
    
    def score_captions_with_clip(self, image, captions):
        """Score captions using CLIP model"""
        inputs = self.clip_processor(text=captions, images=image, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = self.clip_model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)[0].detach().numpy()
        
        return probs
    
    def process_batch(self, batch_paths, batch_idx, total_batches):
        """Process a batch of images"""
        print(f"\nProcessing batch {batch_idx+1}/{total_batches} ({len(batch_paths)} images)")
        
        batch_results = []
        for img_path in tqdm(batch_paths):
            try:
                # Load image
                image = Image.open(img_path).convert('RGB')
                
                # Generate captions
                candidate_captions = self.generate_captions(img_path)
                
                # Score captions with CLIP
                scores = self.score_captions_with_clip(image, candidate_captions)
                
                # Select best caption
                best_idx = np.argmax(scores)
                best_caption = candidate_captions[best_idx]
                best_score = float(scores[best_idx])
                
                # Store all captions and scores
                all_captions_with_scores = [
                    {'caption': caption, 'score': float(score)} 
                    for caption, score in zip(candidate_captions, scores)
                ]
                
                # Add to results
                result = {
                    'image': os.path.basename(img_path),
                    'best_caption': best_caption,
                    'best_score': best_score,
                    'all_captions': all_captions_with_scores
                }
                batch_results.append(result)
                self.results.append(result)
                
                # Respect OpenAI rate limits
                if self.api_call_count % 20 == 0:
                    time.sleep(3)
                    
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
        
        return batch_results
    
    def save_progress(self):
        """Save current progress to files"""
        # Save detailed results with all captions
        detailed_df = pd.DataFrame(self.results)
        detailed_df.to_json(self.output_file.replace('.csv', '_detailed.json'), orient='records')
        
        # Save simplified CSV with just image and best caption
        simple_df = pd.DataFrame([
            {'image': r['image'], 'caption': r['best_caption'], 'score': r['best_score']} 
            for r in self.results
        ])
        simple_df.to_csv(self.output_file, index=False)
        
        print(f"Saved progress: {len(self.results)}/{len(self.image_paths) + len(self.processed_images)} images processed")
        print(f"Estimated API cost so far: ${self.total_cost_estimate:.4f}")
    
    def run(self):
        """Run the caption generation process"""
        total_batches = (len(self.image_paths) + self.batch_size - 1) // self.batch_size
        
        for batch_idx in range(total_batches):
            # Get batch paths
            batch_start = batch_idx * self.batch_size
            batch_end = min((batch_idx + 1) * self.batch_size, len(self.image_paths))
            batch_paths = self.image_paths[batch_start:batch_end]
            
            # Process batch
            self.process_batch(batch_paths, batch_idx, total_batches)
            
            # Save progress
            if (batch_idx + 1) % SAVE_EVERY_N_BATCHES == 0 or batch_idx == total_batches - 1:
                self.save_progress()
        
        print(f"\nCompleted captioning {len(self.results)} images")
        print(f"Total estimated API cost: ${self.total_cost_estimate:.4f}")
        print(f"Results saved to {self.output_file} and {self.output_file.replace('.csv', '_detailed.json')}")


# Run the caption generator
# if __name__ == "__main__":
#     generator = CaptionGenerator(
#         dataset_path=DATASET_PATH,
#         output_file=OUTPUT_FILE,
#         batch_size=BATCH_SIZE,
#         captions_per_image=NUM_CAPTIONS_PER_IMAGE
#     )
#     generator.run()

In [14]:
NUM_CAPTIONS_PER_IMAGE

3

In [19]:
generator = CaptionGenerator(
    dataset_path='/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/raw/Flicker8k_Dataset/test',
    output_file='/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/outputs/naive_test.csv',
    batch_size=16,
    captions_per_image=NUM_CAPTIONS_PER_IMAGE
)
generator.run()

Loading CLIP model...
Found 809 images in /Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/raw/Flicker8k_Dataset/test
Remaining images to process: 809

Processing batch 1/51 (16 images)


100%|██████████| 16/16 [00:14<00:00,  1.11it/s]


Saved progress: 16/809 images processed
Estimated API cost so far: $0.0033

Processing batch 2/51 (16 images)


100%|██████████| 16/16 [00:18<00:00,  1.18s/it]


Saved progress: 32/809 images processed
Estimated API cost so far: $0.0066

Processing batch 3/51 (16 images)


...

Saved progress: 784/809 images processed
Estimated API cost so far: $0.1607

Processing batch 50/51 (16 images)


100%|██████████| 16/16 [00:22<00:00,  1.40s/it]


Saved progress: 800/809 images processed
Estimated API cost so far: $0.1640

Processing batch 51/51 (9 images)


100%|██████████| 9/9 [00:08<00:00,  1.04it/s]

Saved progress: 809/809 images processed
Estimated API cost so far: $0.1658

Completed captioning 809 images
Total estimated API cost: $0.1658
Results saved to /Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/outputs/naive_test.csv and /Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/outputs/naive_test_detailed.json


In [ ]:
# import os
# import glob
# import random
# import shutil
# import pandas as pd

# # Configuration
# DATASET_PATH = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/caption_generator/data/raw/Flicker8k_Dataset/test"
# SAMPLE_SIZE = 300
# SAMPLE_DIR = "sampled_images"
# SAMPLE_FILE = "sampled_images.csv"

# # Create sample directory if it doesn't exist
# os.makedirs(SAMPLE_DIR, exist_ok=True)

# # Get all image paths
# image_paths = glob.glob(os.path.join(DATASET_PATH, "*.jpg"))
# print(f"Found {len(image_paths)} images in {DATASET_PATH}")

# # Randomly sample images
# if len(image_paths) > SAMPLE_SIZE:
#     sampled_paths = random.sample(image_paths, SAMPLE_SIZE)
# else:
#     sampled_paths = image_paths
#     print(f"Warning: Requested {SAMPLE_SIZE} samples but only found {len(image_paths)} images")

# # Copy sampled images to sample directory
# sampled_data = []
# for i, path in enumerate(sampled_paths):
#     image_name = os.path.basename(path)
#     dest_path = os.path.join(SAMPLE_DIR, image_name)
#     shutil.copy(path, dest_path)
#     sampled_data.append({
#         'index': i,
#         'image': image_name,
#         'original_path': path
#     })
    
# # Save sample information
# sample_df = pd.DataFrame(sampled_data)
# sample_df.to_csv(SAMPLE_FILE, index=False)

# print(f"Sampled {len(sampled_paths)} images and copied to {SAMPLE_DIR}")
# print(f"Sample information saved to {SAMPLE_FILE}")